<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch10/section_93.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 93. パープレキシティ
適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

The movie was full of surprises

The movies were full of surprises

The movie were full of surprises

The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

### パープレキシティ（Perplexity）

パープレキシティ（Perplexity）は、
言語モデルが次の単語をどれだけ予測しにくいかを表す評価指標である。

- パープレキシティが 低い 場合
→ モデルは次の単語を高い確率で予測できている

- パープレキシティが 高い 場合
→ 次の単語の候補が多く、予測が難しい

直感的には、
「次の単語が平均して何択に見えているか」
を表す指標と解釈できる。

### 正解率が適さない理由

言語モデルの予測では、次に来る単語が必ずしも一意に定まらない。
そのため、正解・不正解のみで評価する正解率（accuracy）は、
言語モデルの性能評価には適していない。

パープレキシティは、
正解単語に割り当てられた確率の大きさを用いて評価を行うため、
言語モデルに適した指標である。

### エントロピーとの関係

言語モデルは、各単語位置において
「正解単語が出現する確率」を出力する。

エントロピー
$$H(X)$$は、

正解単語の確率に対して

対数を取り、マイナスを付けた値を

全単語で平均したもの

として定義される。
$$
H(X)
=
-\frac{1}{N}
\sum_{i=1}^{N}
\log P(w_i)
$$
この値が大きいほど、モデルは予測に失敗しているといえる。

パープレキシティの定義

エントロピーを人間に読みやすい尺度にするために指数関数で変換したものがパープレキシティである。

$$
\text{Perplexity}(X) = 2^{H(X)}
$$


エントロピーが 0 の場合、
モデルは常に正解を予測できており、パープレキシティは 1 となる。

まとめ

エントロピー
→ 正解単語に対する予測誤差（損失）の平均

パープレキシティ
→ それを「次の単語が何択に見えているか」に変換した指標

パープレキシティが低いほど、言語モデルの性能は高い

In [3]:
!pip install -U transformers

In [15]:
import math
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [5]:
model_id = "gpt2"
sentences = [
    "The movie was full of surprises",
    "The movies were full of surprises",
    "The movie were full of surprises",
    "The movies was full of surprises"
]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id)

In [10]:
inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True)

In [11]:
max_new_tokens = 10
generation_config = GenerationConfig(
    max_new_tokens=max_new_tokens,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)

### nn.CrossEntropyLossのreducionについて
- reuctionには３種類設定できるものがある
  1. mean（デフォルト）
    - 全要素の損失を平均する
    - 返り値はスカラー
  2. sum
    - 全要素の損失を合計
    - 返り値はスカラー
  3. none
    - 何もまとめない
    - 各要素の損失をそのまま返す

今回はわかりやすいように何もしない様に意図的に設定した   

In [13]:
cross_entropy_loss = torch.nn.CrossEntropyLoss(reduction="none")

In [14]:
with torch.no_grad():
  outputs = model(**inputs, generation_config=generation_config)
logits = outputs.logits

shift_input_idsはなぜ最初の1トークンを除外しているのか
- [i, 1:]でi番目のテキストを取得して、最初のトークンを飛ばして取得している。
- これは最初のトークンは文脈を持たず、次単語予測という言語モデルの枠組みでは正解ラベルを定義できないため、パープレキシティ計算から除外される。
shift_logitsはなぜ最後の1トークンを除外しているのか
- logitsは各トークン位置において、次のトークンの予測結果を表している。
- 最後のトークン位置では次に予測すべきトークンが存在しない。

In [16]:
for i, text in enumerate(sentences):
  # contiguous()の必要性
  # https://openillumi.com/pytorch-memory-efficiency-contiguous-method-guide/
  shift_input_ids = inputs.input_ids[i, 1:].contiguous()
  shift_logits = logits[i, :-1, :].contiguous()

  # 正解トークンのtoken_idと書くトークン位置のlogitsを渡し、-logP（正解）を計算する
  H_i = cross_entropy_loss(shift_logits, shift_input_ids)

  # 文内で平均してクロスエントロピーにする
  H = H_i.mean()

  # 自然対数ベースなので、expでPPLに変換する
  ppl = math.e**H

  print(f"""Text: {text}\nPPL: {ppl:.4f}\n""")

Text: The movie was full of surprises
PPL: 99.3539

Text: The movies were full of surprises
PPL: 126.4828

Text: The movie were full of surprises
PPL: 278.8822

Text: The movies was full of surprises
PPL: 274.6658

